In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta, date
#import datetime
from datetime import datetime
import time

### Read the Township metadata once 

In [3]:
csv_input = pd.read_csv("unique_centroid_township_metadata.csv", encoding= 'unicode_escape', low_memory = False)
print(csv_input)
print(csv_input.info())

           fid  InputID   TargetID     Distance
0            1   682187  T022R14W4  4294.801479
1            2   682188  T022R14W4  3884.842191
2            3   682189  T022R14W4  3706.678318
3            4   682190  T022R14W4  3793.114242
4            5   682191  T022R14W4  4127.560216
...        ...      ...        ...          ...
664869  664870    21462  T077R13W6  3571.080156
664870  664871    21463  T077R13W6  4481.194510
664871  664872    21464  T076R13W6  4630.851849
664872  664873    21465  T076R13W6  3658.926218
664873  664874    21466  T076R13W6  2707.239380

[664874 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664874 entries, 0 to 664873
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   fid       664874 non-null  int64  
 1   InputID   664874 non-null  int64  
 2   TargetID  664874 non-null  object 
 3   Distance  664874 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory 

In [4]:
csv_input[csv_input.TargetID == 'T077R13W6'].shape[0]   # around 100 cells will have same value

93

In [5]:
grid_cen  =  csv_input[['TargetID', 'InputID']].copy()
grid_cen = grid_cen.rename( columns = {'InputID':'Grid_id', 'TargetID':'twpid' })
grid_cen

,twpid,Grid_id
0,T022R14W4,682187
1,T022R14W4,682188
2,T022R14W4,682189
3,T022R14W4,682190
4,T022R14W4,682191
...,...,...
664869,T077R13W6,21462
664870,T077R13W6,21463
664871,T076R13W6,21464
664872,T076R13W6,21465


# Read the Townships climate data for every year and merge the Grid ids

In [11]:
# reading the Townships climate data for one year

#********************* Change year here**************************#
year = str(2018)
acis_met_data = pd.read_csv('acis_met_'+str(year)+'.csv', encoding= 'unicode_escape')
print(acis_met_data.shape)

(1476600, 6)


In [12]:
merge_data = pd.merge(grid_cen,acis_met_data,  
                      on = 'twpid',
                      how = 'inner')

print(merge_data)

               twpid  Grid_id        date  Prcp  MaxT  RelHum     Ws
0          T022R14W4   682187  2018-04-01  0.00 -9.68   81.47   7.06
1          T022R14W4   682187  2018-04-02  0.00 -5.19   76.34   9.21
2          T022R14W4   682187  2018-04-03  0.00 -5.57   82.82   9.38
3          T022R14W4   682187  2018-04-04  0.10 -4.75   81.22  12.63
4          T022R14W4   682187  2018-04-05  0.00 -7.69   73.13  16.13
...              ...      ...         ...   ...   ...     ...    ...
142283031  T061R20W5   257318  2018-10-27  0.00  7.36   55.80   3.78
142283032  T061R20W5   257318  2018-10-28  0.00  2.40   77.66   3.42
142283033  T061R20W5   257318  2018-10-29  8.05  5.22   92.80   0.93
142283034  T061R20W5   257318  2018-10-30  0.00  8.52   67.34   2.34
142283035  T061R20W5   257318  2018-10-31  0.58  3.98   77.93   1.83

[142283036 rows x 7 columns]


In [8]:
# count the number of Twpids and grid ids before merging the data
print("grid_cen num of Townships", grid_cen['twpid'].nunique())
print("grid_cen num of grid_ids", grid_cen['Grid_id'].nunique())

print("acis_met_data num of Townships", acis_met_data['twpid'].nunique())

print("Merge data num of Townships", merge_data['twpid'].nunique())
print("Merge data num of grid ids ", merge_data['Grid_id'].nunique())

grid_cen num of Townships 6900
grid_cen num of grid_ids 664874
acis_met_data num of Townships 6900
Merge data num of Townships 6900
Merge data num of grid ids  664874


In [13]:
merge_data = merge_data.drop(columns = ['twpid'])
merge_data

,Grid_id,date,Prcp,MaxT,RelHum,Ws
0,682187,2018-04-01,0.00,-9.68,81.47,7.06
1,682187,2018-04-02,0.00,-5.19,76.34,9.21
2,682187,2018-04-03,0.00,-5.57,82.82,9.38
3,682187,2018-04-04,0.10,-4.75,81.22,12.63
4,682187,2018-04-05,0.00,-7.69,73.13,16.13
...,...,...,...,...,...,...
142283031,257318,2018-10-27,0.00,7.36,55.80,3.78
142283032,257318,2018-10-28,0.00,2.40,77.66,3.42
142283033,257318,2018-10-29,8.05,5.22,92.80,0.93
142283034,257318,2018-10-30,0.00,8.52,67.34,2.34


In [14]:
# Saving to output file
merge_data.to_csv("met_"+year+"_output.csv", index = False)